In [5]:
# Bringing in the data

import pandas as pd

# Read the CSV file into a DataFrame
df_mm = pd.read_csv("C:/Users/hagen/Downloads/NBA DATA/Teams/Transformed Data/mm_data.csv")

df_p = pd.read_csv("C:/Users/hagen/Downloads/NBA DATA/Teams/Transformed Data/p_data.csv")

In [35]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.miscmodels.ordinal_model import OrderedModel
from sklearn.feature_selection import RFE
from statsmodels.stats.outliers_influence import variance_inflation_factor

def multi_ordinal_top5(df, selected_features, link):
    """
    Custom cross-validation for predicting NBA champion using Ordinal Regression (OrderedModel),
    with group information to rank the teams by their predicted level (score).
    """
    results = []

    # Iterate over the years you want to predict (2014 to 2023)
    for test_year in range(2014, 2024):
        
        # Training data: the previous 5 years of data (e.g., 2009-2013 for 2014 prediction)
        train_data = df[df['Year'].isin(range(test_year - 5, test_year))]
        
        # Test data: current year
        test_data = df[df['Year'] == test_year]
        
        # Prepare features and target for training and testing
        X_train = train_data[selected_features]
        y_train = train_data['Levels']
        
        X_test = test_data[selected_features]
        y_test = test_data['Levels']

        # If more than 1 feature is selected, calculate VIF
        if len(selected_features) > 1:
            vif_data = pd.DataFrame()
            vif_data["Feature"] = X_train.columns
            vif_data["VIF"] = [variance_inflation_factor(X_train.values, i) for i in range(X_train.shape[1])]
            print(vif_data)
        
        # Train the model using Ordinal Regression (OrderedModel)
        model = OrderedModel(y_train, X_train, distr=link)
        result = model.fit(maxiter=1000)

        # Make predictions on the test data
        y_pred = result.predict(X_test)

        # Rank teams based on predicted scores (we take the predicted probability of the highest class)
        ranked_teams = test_data[['Team']].copy()
        ranked_teams['Score'] = y_pred.iloc[:, 4]  # Take the probability of the highest class
        ranked_teams = ranked_teams.sort_values(by='Score', ascending=False)
        
        # Get the actual champion team (level 4) from the test data
        actual_champion = test_data[test_data['Levels'] == 4]['Team'].values[0]

        # Get top 5 teams and their scores
        top_teams = ranked_teams.head(5)
        top_teams_list = top_teams.apply(lambda row: f"{row['Team']}: {row['Score']:.4f}", axis=1).tolist()
        
        # Store results
        results.append([test_year, actual_champion] + top_teams_list)
    
    # Create dataframe with results
    columns = ['Year', 'Actual Champion', 'First Pick', 'Second Pick', 'Third Pick', 'Fourth Pick', 'Fifth Pick']
    results_df = pd.DataFrame(results, columns=columns)
    results_df = results_df.sort_values(by="Year", ascending=False)

    # Compute accuracy
    accuracy = (results_df['Actual Champion'] == results_df['First Pick'].str.split(': ').str[0]).mean()
    print(f"Model Accuracy: {accuracy * 100:.2f}%")
    
    return results_df


In [ ]:
# all linking options: 
#Logit: Most commonly used, maps linear combination of features to probabilities with log-odds.
#Probit: Uses the CDF of the normal distribution, often used when normally distributed errors are assumed.
#Cauchit:Uses the inverse tangent (arctangent) function.
#Cloglog: Complementary log-log, used for extreme probabilities.
#Log-Log: Similar to cloglog, but for extreme probabilities near 1.
#Identity: No transformation applied (rarely used for ordinal regression).

In [37]:
selected_features = ['W', 'eFG%']

multi_ordinal_top5(df_mm, selected_features, 'logit')

  Feature     VIF
0       W  7.7102
1    eFG%  7.7102
Optimization terminated successfully.
         Current function value: 0.755511
         Iterations: 646
         Function evaluations: 987
  Feature       VIF
0       W  8.250266
1    eFG%  8.250266
Optimization terminated successfully.
         Current function value: 0.747736
         Iterations: 598
         Function evaluations: 918
  Feature       VIF
0       W  9.081993
1    eFG%  9.081993
Optimization terminated successfully.
         Current function value: 0.761575
         Iterations: 631
         Function evaluations: 967
  Feature       VIF
0       W  7.717541
1    eFG%  7.717541
Optimization terminated successfully.
         Current function value: 0.704521
         Iterations: 561
         Function evaluations: 869
  Feature      VIF
0       W  7.93086
1    eFG%  7.93086
Optimization terminated successfully.
         Current function value: 0.679001
         Iterations: 576
         Function evaluations: 890
  Feature

,Year,Actual Champion,First Pick,Second Pick,Third Pick,Fourth Pick,Fifth Pick
9,2023,Boston Celtics,Boston Celtics: 0.4670,Oklahoma City Thunder: 0.1172,Denver Nuggets: 0.1057,Minnesota Timberwolves: 0.0810,Los Angeles Clippers: 0.0237
8,2022,Denver Nuggets,Milwaukee Bucks: 0.4877,Boston Celtics: 0.4203,Philadelphia 76ers: 0.1962,Denver Nuggets: 0.1558,Cleveland Cavaliers: 0.0739
7,2021,Golden State Warriors,Phoenix Suns: 0.4901,Memphis Grizzlies: 0.0715,Miami Heat: 0.0182,Golden State Warriors: 0.0170,Dallas Mavericks: 0.0144
6,2020,Milwaukee Bucks,Utah Jazz: 0.5599,Phoenix Suns: 0.4452,Philadelphia 76ers: 0.2676,Brooklyn Nets: 0.1615,Denver Nuggets: 0.1187
5,2019,Los Angeles Lakers,Milwaukee Bucks: 0.6179,Toronto Raptors: 0.2806,Los Angeles Lakers: 0.2196,Los Angeles Clippers: 0.0709,Boston Celtics: 0.0461
4,2018,Toronto Raptors,Milwaukee Bucks: 0.5817,Golden State Warriors: 0.4120,Toronto Raptors: 0.3734,Houston Rockets: 0.0947,Denver Nuggets: 0.0946
3,2017,Golden State Warriors,Houston Rockets: 0.5563,Toronto Raptors: 0.1392,Golden State Warriors: 0.1097,Boston Celtics: 0.0386,Philadelphia 76ers: 0.0149
2,2016,Golden State Warriors,Golden State Warriors: 0.4354,San Antonio Spurs: 0.1315,Houston Rockets: 0.0245,Boston Celtics: 0.0146,Toronto Raptors: 0.0084
1,2015,Cleveland Cavaliers,Golden State Warriors: 0.3706,San Antonio Spurs: 0.1627,Cleveland Cavaliers: 0.0212,Toronto Raptors: 0.0192,Oklahoma City Thunder: 0.0138
0,2014,Golden State Warriors,Golden State Warriors: 0.3221,Atlanta Hawks: 0.0746,Houston Rockets: 0.0299,Memphis Grizzlies: 0.0271,Los Angeles Clippers: 0.0258


In [39]:
selected_features = ['W', 'eFG%']

multi_ordinal_top5(df_mm, selected_features, 'probit')

  Feature     VIF
0       W  7.7102
1    eFG%  7.7102
Optimization terminated successfully.
         Current function value: 0.747081
         Iterations: 520
         Function evaluations: 792
  Feature       VIF
0       W  8.250266
1    eFG%  8.250266
Optimization terminated successfully.
         Current function value: 0.739378
         Iterations: 496
         Function evaluations: 752
  Feature       VIF
0       W  9.081993
1    eFG%  9.081993
Optimization terminated successfully.
         Current function value: 0.754265
         Iterations: 584
         Function evaluations: 894
  Feature       VIF
0       W  7.717541
1    eFG%  7.717541
Optimization terminated successfully.
         Current function value: 0.695018
         Iterations: 620
         Function evaluations: 942
  Feature      VIF
0       W  7.93086
1    eFG%  7.93086
Optimization terminated successfully.
         Current function value: 0.671006
         Iterations: 561
         Function evaluations: 871
  Feature

,Year,Actual Champion,First Pick,Second Pick,Third Pick,Fourth Pick,Fifth Pick
9,2023,Boston Celtics,Boston Celtics: 0.5119,Oklahoma City Thunder: 0.1450,Denver Nuggets: 0.1308,Minnesota Timberwolves: 0.0983,Los Angeles Clippers: 0.0209
8,2022,Denver Nuggets,Milwaukee Bucks: 0.5356,Boston Celtics: 0.4807,Philadelphia 76ers: 0.2470,Denver Nuggets: 0.2039,Cleveland Cavaliers: 0.0895
7,2021,Golden State Warriors,Phoenix Suns: 0.5356,Memphis Grizzlies: 0.0767,Miami Heat: 0.0147,Golden State Warriors: 0.0138,Dallas Mavericks: 0.0097
6,2020,Milwaukee Bucks,Utah Jazz: 0.6096,Phoenix Suns: 0.5051,Philadelphia 76ers: 0.3183,Brooklyn Nets: 0.2124,Denver Nuggets: 0.1518
5,2019,Los Angeles Lakers,Milwaukee Bucks: 0.6791,Toronto Raptors: 0.3466,Los Angeles Lakers: 0.2840,Los Angeles Clippers: 0.0896,Boston Celtics: 0.0530
4,2018,Toronto Raptors,Milwaukee Bucks: 0.6131,Golden State Warriors: 0.4565,Toronto Raptors: 0.4203,Denver Nuggets: 0.1142,Houston Rockets: 0.1142
3,2017,Golden State Warriors,Houston Rockets: 0.5752,Toronto Raptors: 0.1612,Golden State Warriors: 0.1211,Boston Celtics: 0.0373,Philadelphia 76ers: 0.0091
2,2016,Golden State Warriors,Golden State Warriors: 0.4524,San Antonio Spurs: 0.1493,Houston Rockets: 0.0184,Boston Celtics: 0.0087,Toronto Raptors: 0.0035
1,2015,Cleveland Cavaliers,Golden State Warriors: 0.3626,San Antonio Spurs: 0.1678,Cleveland Cavaliers: 0.0132,Toronto Raptors: 0.0117,Oklahoma City Thunder: 0.0067
0,2014,Golden State Warriors,Golden State Warriors: 0.3185,Atlanta Hawks: 0.0697,Houston Rockets: 0.0222,Memphis Grizzlies: 0.0209,Los Angeles Clippers: 0.0167


In [45]:
import scipy.stats as stats

selected_features = ['W', 'eFG%']

multi_ordinal_top5(df_mm, selected_features, stats.cauchy)

  Feature     VIF
0       W  7.7102
1    eFG%  7.7102
Optimization terminated successfully.
         Current function value: 0.832153
         Iterations: 818
         Function evaluations: 1255
  Feature       VIF
0       W  8.250266
1    eFG%  8.250266
Optimization terminated successfully.
         Current function value: 0.830606
         Iterations: 815
         Function evaluations: 1225
  Feature       VIF
0       W  9.081993
1    eFG%  9.081993
Optimization terminated successfully.
         Current function value: 0.842325
         Iterations: 670
         Function evaluations: 1041
  Feature       VIF
0       W  7.717541
1    eFG%  7.717541
Optimization terminated successfully.
         Current function value: 0.792842
         Iterations: 693
         Function evaluations: 1082
  Feature      VIF
0       W  7.93086
1    eFG%  7.93086
Optimization terminated successfully.
         Current function value: 0.759408
         Iterations: 760
         Function evaluations: 1166
  Fe

,Year,Actual Champion,First Pick,Second Pick,Third Pick,Fourth Pick,Fifth Pick
9,2023,Boston Celtics,Boston Celtics: 0.0680,Oklahoma City Thunder: 0.0472,Denver Nuggets: 0.0466,Minnesota Timberwolves: 0.0445,Los Angeles Clippers: 0.0371
8,2022,Denver Nuggets,Milwaukee Bucks: 0.0743,Boston Celtics: 0.0674,Philadelphia 76ers: 0.0534,Denver Nuggets: 0.0498,Memphis Grizzlies: 0.0445
7,2021,Golden State Warriors,Phoenix Suns: 0.1451,Memphis Grizzlies: 0.0667,Miami Heat: 0.0471,Golden State Warriors: 0.0462,Dallas Mavericks: 0.0460
6,2020,Milwaukee Bucks,Utah Jazz: 0.2016,Phoenix Suns: 0.1552,Philadelphia 76ers: 0.1212,Brooklyn Nets: 0.0864,Denver Nuggets: 0.0822
5,2019,Los Angeles Lakers,Milwaukee Bucks: 0.3586,Toronto Raptors: 0.1642,Los Angeles Lakers: 0.1337,Los Angeles Clippers: 0.0871,Boston Celtics: 0.0780
4,2018,Toronto Raptors,Milwaukee Bucks: 0.4423,Golden State Warriors: 0.2476,Toronto Raptors: 0.2356,Denver Nuggets: 0.1037,Houston Rockets: 0.1015
3,2017,Golden State Warriors,Houston Rockets: 0.5460,Toronto Raptors: 0.1414,Golden State Warriors: 0.1240,Boston Celtics: 0.0859,Philadelphia 76ers: 0.0666
2,2016,Golden State Warriors,Golden State Warriors: 0.3697,San Antonio Spurs: 0.1226,Houston Rockets: 0.0737,Boston Celtics: 0.0635,Cleveland Cavaliers: 0.0575
1,2015,Cleveland Cavaliers,Golden State Warriors: 0.3381,San Antonio Spurs: 0.1550,Cleveland Cavaliers: 0.0725,Toronto Raptors: 0.0698,Oklahoma City Thunder: 0.0654
0,2014,Golden State Warriors,Golden State Warriors: 0.2633,Atlanta Hawks: 0.1020,Los Angeles Clippers: 0.0745,Houston Rockets: 0.0741,San Antonio Spurs: 0.0695


In [59]:
selected_features = ['W', 'eFG%']

multi_ordinal_top5(df_p, selected_features, 'logit')

  Feature       VIF
0       W  6.048703
1    eFG%  6.048703
Optimization terminated successfully.
         Current function value: 0.704803
         Iterations: 557
         Function evaluations: 851
  Feature       VIF
0       W  6.736676
1    eFG%  6.736676
Optimization terminated successfully.
         Current function value: 0.698136
         Iterations: 575
         Function evaluations: 886
  Feature       VIF
0       W  6.500348
1    eFG%  6.500348
Optimization terminated successfully.
         Current function value: 0.674890
         Iterations: 606
         Function evaluations: 947
  Feature       VIF
0       W  6.474259
1    eFG%  6.474259
Optimization terminated successfully.
         Current function value: 0.620588
         Iterations: 655
         Function evaluations: 1010
  Feature       VIF
0       W  6.925942
1    eFG%  6.925942
Optimization terminated successfully.
         Current function value: 0.584939
         Iterations: 573
         Function evaluations: 901

,Year,Actual Champion,First Pick,Second Pick,Third Pick,Fourth Pick,Fifth Pick
9,2023,Boston Celtics,Boston Celtics: 0.3778,Oklahoma City Thunder: 0.2308,Denver Nuggets: 0.2238,Minnesota Timberwolves: 0.1242,Los Angeles Clippers: 0.0830
8,2022,Denver Nuggets,Milwaukee Bucks: 0.3849,Boston Celtics: 0.3038,Philadelphia 76ers: 0.1907,Denver Nuggets: 0.1372,Cleveland Cavaliers: 0.0510
7,2021,Golden State Warriors,Phoenix Suns: 0.4410,Memphis Grizzlies: 0.1950,Golden State Warriors: 0.1525,Miami Heat: 0.1448,Dallas Mavericks: 0.0502
6,2020,Milwaukee Bucks,Utah Jazz: 0.4471,Phoenix Suns: 0.3124,Philadelphia 76ers: 0.1277,Brooklyn Nets: 0.1230,Los Angeles Clippers: 0.0452
5,2019,Los Angeles Lakers,Milwaukee Bucks: 0.4529,Toronto Raptors: 0.2432,Los Angeles Lakers: 0.1581,Los Angeles Clippers: 0.0747,Boston Celtics: 0.0304
4,2018,Toronto Raptors,Milwaukee Bucks: 0.3925,Toronto Raptors: 0.2660,Golden State Warriors: 0.1846,Denver Nuggets: 0.0750,Houston Rockets: 0.0498
3,2017,Golden State Warriors,Houston Rockets: 0.3477,Toronto Raptors: 0.2487,Golden State Warriors: 0.1514,Boston Celtics: 0.1216,Philadelphia 76ers: 0.0644
2,2016,Golden State Warriors,Golden State Warriors: 0.3243,San Antonio Spurs: 0.2430,Houston Rockets: 0.1614,Boston Celtics: 0.1120,Toronto Raptors: 0.0388
1,2015,Cleveland Cavaliers,Golden State Warriors: 0.2842,San Antonio Spurs: 0.2063,Cleveland Cavaliers: 0.1500,Toronto Raptors: 0.1293,Oklahoma City Thunder: 0.0658
0,2014,Golden State Warriors,Golden State Warriors: 0.2914,Atlanta Hawks: 0.2188,Houston Rockets: 0.1399,Los Angeles Clippers: 0.1214,Memphis Grizzlies: 0.0947


In [61]:
selected_features = ['W', 'eFG%']

multi_ordinal_top5(df_p, selected_features, 'probit')

  Feature       VIF
0       W  6.048703
1    eFG%  6.048703
Optimization terminated successfully.
         Current function value: 0.699571
         Iterations: 509
         Function evaluations: 784
  Feature       VIF
0       W  6.736676
1    eFG%  6.736676
Optimization terminated successfully.
         Current function value: 0.692418
         Iterations: 588
         Function evaluations: 910
  Feature       VIF
0       W  6.500348
1    eFG%  6.500348
Optimization terminated successfully.
         Current function value: 0.668205
         Iterations: 495
         Function evaluations: 764
  Feature       VIF
0       W  6.474259
1    eFG%  6.474259
Optimization terminated successfully.
         Current function value: 0.610817
         Iterations: 509
         Function evaluations: 781
  Feature       VIF
0       W  6.925942
1    eFG%  6.925942
Optimization terminated successfully.
         Current function value: 0.576676
         Iterations: 636
         Function evaluations: 967


,Year,Actual Champion,First Pick,Second Pick,Third Pick,Fourth Pick,Fifth Pick
9,2023,Boston Celtics,Boston Celtics: 0.4032,Oklahoma City Thunder: 0.2626,Denver Nuggets: 0.2585,Minnesota Timberwolves: 0.1493,Los Angeles Clippers: 0.0980
8,2022,Denver Nuggets,Milwaukee Bucks: 0.4109,Boston Celtics: 0.3337,Philadelphia 76ers: 0.2183,Denver Nuggets: 0.1585,Cleveland Cavaliers: 0.0534
7,2021,Golden State Warriors,Phoenix Suns: 0.4615,Memphis Grizzlies: 0.2225,Golden State Warriors: 0.1792,Miami Heat: 0.1702,Dallas Mavericks: 0.0542
6,2020,Milwaukee Bucks,Utah Jazz: 0.4424,Phoenix Suns: 0.3180,Philadelphia 76ers: 0.1309,Brooklyn Nets: 0.1263,Los Angeles Clippers: 0.0389
5,2019,Los Angeles Lakers,Milwaukee Bucks: 0.4441,Toronto Raptors: 0.2485,Los Angeles Lakers: 0.1614,Los Angeles Clippers: 0.0708,Boston Celtics: 0.0223
4,2018,Toronto Raptors,Milwaukee Bucks: 0.3770,Toronto Raptors: 0.2609,Golden State Warriors: 0.1827,Denver Nuggets: 0.0675,Houston Rockets: 0.0415
3,2017,Golden State Warriors,Houston Rockets: 0.3364,Toronto Raptors: 0.2431,Golden State Warriors: 0.1489,Boston Celtics: 0.1142,Philadelphia 76ers: 0.0560
2,2016,Golden State Warriors,Golden State Warriors: 0.3218,San Antonio Spurs: 0.2397,Houston Rockets: 0.1639,Boston Celtics: 0.1097,Toronto Raptors: 0.0306
1,2015,Cleveland Cavaliers,Golden State Warriors: 0.2933,San Antonio Spurs: 0.2166,Cleveland Cavaliers: 0.1571,Toronto Raptors: 0.1300,Oklahoma City Thunder: 0.0642
0,2014,Golden State Warriors,Golden State Warriors: 0.3024,Atlanta Hawks: 0.2304,Houston Rockets: 0.1464,Los Angeles Clippers: 0.1292,Memphis Grizzlies: 0.0908


In [63]:
import scipy.stats as stats

selected_features = ['W', 'eFG%']

multi_ordinal_top5(df_p, selected_features, stats.cauchy)

  Feature       VIF
0       W  6.048703
1    eFG%  6.048703
Optimization terminated successfully.
         Current function value: 0.775417
         Iterations: 759
         Function evaluations: 1183
  Feature       VIF
0       W  6.736676
1    eFG%  6.736676
Optimization terminated successfully.
         Current function value: 0.767947
         Iterations: 784
         Function evaluations: 1219
  Feature       VIF
0       W  6.500348
1    eFG%  6.500348


C:\Users\hagen\anaconda3\Lib\site-packages\statsmodels\base\optimizer.py:736: RuntimeWarning: Maximum number of iterations has been exceeded.
  retvals = optimize.fmin(f, start_params, args=fargs, xtol=xtol,
C:\Users\hagen\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


  Feature       VIF
0       W  6.474259
1    eFG%  6.474259
Optimization terminated successfully.
         Current function value: 0.708116
         Iterations: 984
         Function evaluations: 1514
  Feature       VIF
0       W  6.925942
1    eFG%  6.925942
Optimization terminated successfully.
         Current function value: 0.659133
         Iterations: 766
         Function evaluations: 1182
  Feature       VIF
0       W  7.140512
1    eFG%  7.140512
Optimization terminated successfully.
         Current function value: 0.582981
         Iterations: 831
         Function evaluations: 1255
  Feature       VIF
0       W  6.862114
1    eFG%  6.862114
Optimization terminated successfully.
         Current function value: 0.595578
         Iterations: 739
         Function evaluations: 1143
  Feature       VIF
0       W  7.314591
1    eFG%  7.314591
Optimization terminated successfully.
         Current function value: 0.615285
         Iterations: 751
         Function evaluations: 

,Year,Actual Champion,First Pick,Second Pick,Third Pick,Fourth Pick,Fifth Pick
9,2023,Boston Celtics,Boston Celtics: 0.0781,Oklahoma City Thunder: 0.0655,Denver Nuggets: 0.0654,Minnesota Timberwolves: 0.0563,Los Angeles Clippers: 0.0516
8,2022,Denver Nuggets,Milwaukee Bucks: 0.1329,Boston Celtics: 0.1164,Philadelphia 76ers: 0.0929,Denver Nuggets: 0.0834,Cleveland Cavaliers: 0.0625
7,2021,Golden State Warriors,Phoenix Suns: 0.2682,Memphis Grizzlies: 0.1276,Golden State Warriors: 0.1139,Miami Heat: 0.1109,Dallas Mavericks: 0.0748
6,2020,Milwaukee Bucks,Utah Jazz: 0.3425,Phoenix Suns: 0.2160,Brooklyn Nets: 0.1137,Philadelphia 76ers: 0.1085,Los Angeles Clippers: 0.0766
5,2019,Los Angeles Lakers,Milwaukee Bucks: 0.4481,Toronto Raptors: 0.1896,Los Angeles Lakers: 0.1388,Los Angeles Clippers: 0.0938,Boston Celtics: 0.0681
4,2018,Toronto Raptors,Milwaukee Bucks: 0.4752,Toronto Raptors: 0.2746,Golden State Warriors: 0.1917,Denver Nuggets: 0.1031,Houston Rockets: 0.0903
3,2017,Golden State Warriors,Houston Rockets: 0.4069,Toronto Raptors: 0.2734,Golden State Warriors: 0.1770,Boston Celtics: 0.1555,Philadelphia 76ers: 0.1117
2,2016,Golden State Warriors,Golden State Warriors: 0.3086,San Antonio Spurs: 0.2328,Houston Rockets: 0.1634,Boston Celtics: 0.1334,Toronto Raptors: 0.0863
1,2015,Cleveland Cavaliers,Golden State Warriors: 0.1811,San Antonio Spurs: 0.1435,Cleveland Cavaliers: 0.1209,Toronto Raptors: 0.1184,Oklahoma City Thunder: 0.0855
0,2014,Golden State Warriors,Golden State Warriors: 0.1900,Atlanta Hawks: 0.1531,Houston Rockets: 0.1195,Los Angeles Clippers: 0.1103,Memphis Grizzlies: 0.1036
